<a href="https://colab.research.google.com/github/mmagnus/mmagnus/blob/main/nanoGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
! wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-09-04 14:18:55--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.04s   

2025-09-04 14:18:55 (24.1 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [ ]:
with open ('input.txt', 'r', encoding='utf-8') as f:
  text = f. read()
print("length of dataset in characters: ", len(text) )

# let's look at the first 1000 characters
print(text[:1000])

length of dataset in characters:  1115394
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hung

In [ ]:
chars = sorted (list (set (text) ))
vocab_size = len (chars)
print(''. join (chars))
print (vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [ ]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate (chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string
print (encode ("hii there") )
print(decode (encode("hii there") ))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [ ]:
# let's now encode the entire text dataset and store it into a torch. Tensor
import torch # we use PyTorch: https://pytorch.org
data = torch. tensor (encode (text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:10]) # the 1000 characters we looked at earier will to the GPT look like this
print(text[:10])
for x, y in zip(data[:15], text[:15]):
  print(y, str(x))

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47])
First Citi
F tensor(18)
i tensor(47)
r tensor(56)
s tensor(57)
t tensor(58)
  tensor(1)
C tensor(15)
i tensor(47)
t tensor(58)
i tensor(47)
z tensor(64)
e tensor(43)
n tensor(52)
: tensor(10)

 tensor(0)


In [ ]:
n = int (0.9*len(data)) # first 90% will be train, rest val
train_data = data [: n]
val_data = data [n: ]
block_size = 8